In [ ]:
Цель: "similarity sentences", найти и сопоставить разные по написанию, но одинаковые по смыслу предложения
Задачи:
    1. провести очистку данных от символов, регистра, убрать предложения, состоящие из пары слов(оптимизация)
    (пока для одной колонки, самой инфорнмативной "body_text")
    2. обучить модель bert на очищенных данных и сопоставить предложения с помощью cousine similarity
    3. визуализировать полученные результаты

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("D:\\wikibooks dataset\\ru-books-dataset.csv")#поменять адресс
df.head()

,title,url,abstract,body_text,body_html
0,Викиучебник: Техника и технология средств масс...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,* [станция|Рабочая станция];,Рабочая станция;\nСервер;\nПерсональный компью...,"<div class=""mw-parser-output""><ul><li><a href=..."
1,Викиучебник: АОН/Пилотское свидетельство,https://ru.wikibooks.org/wiki/%D0%90%D0%9E%D0%...,Гражданское пилотское свидетельство - разрешен...,В Википедии имеется статья по теме «Свидетельс...,"<div class=""mw-parser-output""><div class=""info..."
2,Викиучебник: Книга программиста/Структуры данн...,https://ru.wikibooks.org/wiki/%D0%9A%D0%BD%D0%...,К оглавлению,"К оглавлению\nВсе программы, код которых вылож...","<div class=""mw-parser-output""><p><a href=""/wik..."
3,Викиучебник: Тесты НМО/Гигиенические основы и ...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,Гигиенические основы и медицинский контроль за...,Гигиенические основы и медицинский контроль за...,"<div class=""mw-parser-output""><p><b>Гигиеничес..."
4,Викиучебник: Коктейли/Пенная фея,https://ru.wikibooks.org/wiki/%D0%9A%D0%BE%D0%...,Пенная фея,Пенная фея\n\nДжин Old Tom — 60 г\nАбсент — 15...,"<div class=""mw-parser-output""><p><b>Пенная фея..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7898 entries, 0 to 7897
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      7898 non-null   object
 1   url        7898 non-null   object
 2   abstract   7840 non-null   object
 3   body_text  7880 non-null   object
 4   body_html  7898 non-null   object
dtypes: object(5)
memory usage: 308.6+ KB


In [4]:
df.shape

(7898, 5)

In [5]:
df.isnull()

,title,url,abstract,body_text,body_html
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
7893,False,False,False,False,False
7894,False,False,False,False,False
7895,False,False,False,False,False
7896,False,False,True,True,False


In [3]:
data = df[[ "title", "abstract", "body_text"]] #убрать ненужные колонки

data.head()


,title,abstract,body_text
0,Викиучебник: Техника и технология средств масс...,* [станция|Рабочая станция];,Рабочая станция;\nСервер;\nПерсональный компью...
1,Викиучебник: АОН/Пилотское свидетельство,Гражданское пилотское свидетельство - разрешен...,В Википедии имеется статья по теме «Свидетельс...
2,Викиучебник: Книга программиста/Структуры данн...,К оглавлению,"К оглавлению\nВсе программы, код которых вылож..."
3,Викиучебник: Тесты НМО/Гигиенические основы и ...,Гигиенические основы и медицинский контроль за...,Гигиенические основы и медицинский контроль за...
4,Викиучебник: Коктейли/Пенная фея,Пенная фея,Пенная фея\n\nДжин Old Tom — 60 г\nАбсент — 15...


In [10]:
print(data["body_text"][1:2].value_counts())

В Википедии имеется статья по теме «Свидетельство частного пилота»\n\n\nГражданское пилотское свидетельство - разрешение на управление определенным видом воздушного судна. Внутри этого свидетельства может быть много разных отметок (а может и не быть), которые или урезают или увеличивают количество разрешенных функций. Самое понятное и известное массам это свидетельство частного пилота. Его выдают после обучения на самолете или на вертолете. Для краткости в разговорной речи это свидетельство могут обозначать через название его иностранного аналога - ППЛ или ПиПиЭль (PPL).  Раньше это свидетельство в России называлось свидетельство пилота-любителя.\nПорядок выдачи свидетельств описан в ФАП-147. Содержимое этого документа почти целиком повторяет Приложение 1 "Выдача свидетельств авиационному персоналу" к Чикагской конвенции. За исключением того что свидетельства СВС в нем нет и это российская особенность.\nПилотские в каждой стране являются национальными. Выдает их CAA. В разных странах э

In [4]:
print(len(data["body_text"]))

7898


In [ ]:
Очистка данных

In [5]:
import re
def clear_text(text):
    cleaned = re.sub(r"(\\n|\n|\r)", " ", text) # expected string or bytes-like object
    cleaned = re.sub(r"\s{2,}", " ", cleaned)
    lower = cleaned.lower()
    sentences = re.split('\.', lower) #если различать точки как окончание предложений, 
    if len(sentences) < 4:
        return ""
        
        #убрав сокращения, то можно повысить точность 
    return list(filter(None, sentences))

d = [ clear_text(str(row)) for row in data["body_text"]]
     
flat_list = [item for sublist in d for item in sublist]
print(flat_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
flat_list = [] #циклы для перехода к виду ["...", "..."] вместо [["..."], ["..."]]
for sublist in d:
    for item in sublist:
        flat_list.append(item)

In [7]:
flat_list

['в википедии имеется статья по теме «свидетельство частного пилота» гражданское пилотское свидетельство - разрешение на управление определенным видом воздушного судна',
 ' внутри этого свидетельства может быть много разных отметок (а может и не быть), которые или урезают или увеличивают количество разрешенных функций',
 ' самое понятное и известное массам это свидетельство частного пилота',
 ' его выдают после обучения на самолете или на вертолете',
 ' для краткости в разговорной речи это свидетельство могут обозначать через название его иностранного аналога - ппл или пипиэль (ppl)',
 ' раньше это свидетельство в россии называлось свидетельство пилота-любителя',
 ' порядок выдачи свидетельств описан в фап-147',
 ' содержимое этого документа почти целиком повторяет приложение 1 "выдача свидетельств авиационному персоналу" к чикагской конвенции',
 ' за исключением того что свидетельства свс в нем нет и это российская особенность',
 ' пилотские в каждой стране являются национальными',
 '

In [20]:
len(flat_list)

522004

In [8]:
len(flat_list)#оптимизация при удалении предложений меньше 4 символов примерно 3000

519653

In [ ]:
Загрузка модели bert

In [9]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in c:\users\panao\anaconda3\lib\site-packages (2.1.0)


In [10]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [67]:
много по времени, нужно убрать неинформативные предложения


SyntaxError: invalid syntax (<ipython-input-67-90012c147c76>, line 1)

In [12]:
%%time
text_embeddings = model.encode(flat_list[:1000], batch_size = 8, show_progress_bar = True)


Wall time: 1min 8s


In [13]:
%%time
import numpy as np
np.shape(text_embeddings)

Wall time: 998 µs


(1000, 768)

In [14]:
%%time
similarities = cosine_similarity(text_embeddings)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.0000002  0.9640744  0.96396065 ... 0.95453465 0.94263685 0.9214083 ]
 [0.9640744  0.99999964 0.93593544 ... 0.9422665  0.93110996 0.9298673 ]
 [0.96396065 0.93593544 1.0000001  ... 0.928115   0.9316046  0.908844  ]
 ...
 [0.95453465 0.9422665  0.928115   ... 0.9999997  0.94380367 0.92276746]
 [0.94263685 0.93110996 0.9316046  ... 0.94380367 0.99999994 0.9449273 ]
 [0.9214083  0.9298673  0.908844   ... 0.92276746 0.9449273  0.9999999 ]]

Wall time: 14 ms


In [15]:
similarities_sorted = similarities.argsort()

In [16]:
similarities_sorted

array([[986, 788, 799, ...,  14, 853,   0],
       [986, 409, 138, ...,  19,  24,   1],
       [986, 985, 980, ...,  53,   0,   2],
       ...,
       [986, 980, 424, ..., 851, 783, 997],
       [986, 980, 985, ..., 909,  52, 998],
       [986, 985, 409, ..., 909, 991, 999]], dtype=int64)

In [17]:
id_1 = []
id_2 = []
score = []
for index,array in enumerate(similarities_sorted):
    id_1.append(flat_list[index])
    id_2.append(flat_list[array[-2]])
    score.append(similarities[index][array[-2]])
index_df = pd.DataFrame({'id_1' : id_1,
                          'id_2' : id_2,
                          'score' : score})

In [18]:
index_df

,id_1,id_2,score
0,в википедии имеется статья по теме «свидетельс...,профилактика пролежней» назовите объект станд...,0.984509
1,внутри этого свидетельства может быть много р...,государственные учебные заведения этой услуги...,0.973025
2,самое понятное и известное массам это свидете...,в википедии имеется статья по теме «свидетельс...,0.963961
3,его выдают после обучения на самолете или на ...,у современного свидетельства пилота срока нет...,0.959501
4,для краткости в разговорной речи это свидетел...,фгис рап - база пилотских[править] с определе...,0.981835
...,...,...,...
995,последние месячные начались 10 дней назад,2630-10 «санитарно-эпидемиологические требован...,0.915807
996,цикл регулярный с 14 лет,содержание 1 рисование линий 1,0.840938
997,"в анамнезе киста правого яичника, разрешившая...","слегка все размешиваем, после чего добавляем ...",0.963378
998,хронические заболевания отрицает,"когда он заканчивался, требовалось ""продление""",0.961966


In [54]:
index_df.tail(20)


,id_1,id_2,score
60,"старые свидетельства, выданны фла россии, мог...",раньше это свидетельство в россии называлось ...,0.978295
61,как выдаются пилотские[править] пилотское сви...,какие пилотские бывают[править] свидетельства...,0.979575
62,н,см,0.769943
63,"""представления"", которое готовит ауц (авиацио...","такие авиационно-учебные центра (ауц), которы...",0.980015
64,фгис рап - база пилотских[править] с определе...,для краткости в разговорной речи это свидетел...,0.981835
65,"""реестр выданных свидетельств авиационного пе...",его действительность подтверждается двумя док...,0.972742
66,о ней написано на фгис рап,"к таким странам относится, например, сербия",0.919646
67,документы выпускника ауц[править] какие докум...,также[править] аон/ауц аон/пилотское свидетел...,0.973823
68,"с пилотским свидетельством ситуация, к сожале...",сейчас эти свидетельства по-прежнему действую...,0.979643
69,тут могут попросить все документы,вот они имеют срок действия,0.946667


In [19]:
index_df.score.unique()

array([0.98450899, 0.9730252 , 0.96396065, 0.95950133, 0.98183548,
       0.97829533, 0.9152413 , 0.9780423 , 0.9747138 , 0.96723986,
       0.85736996, 0.95482326, 0.95737761, 0.96649462, 0.98409283,
       0.95793104, 0.97387469, 0.97539902, 0.96958506, 0.97943938,
       0.97653472, 0.94098753, 0.97988355, 0.98001522, 0.98187864,
       0.97903818, 0.97957444, 0.97982478, 0.9298287 , 0.97884047,
       0.98286772, 0.97601819, 0.97063422, 0.96645617, 0.96091658,
       0.97187096, 0.9447524 , 0.96142638, 0.95380539, 0.91896969,
       0.97960383, 0.97733033, 0.96735531, 0.93568623, 0.96011394,
       0.96315706, 0.97459841, 0.98041821, 0.95360154, 0.96196562,
       0.97471714, 0.95036268, 0.92545205, 0.97216177, 0.84599268,
       0.97274137, 0.93101835, 0.97382337, 0.97979426, 0.94666684,
       0.92014158, 0.96068603, 0.9706744 , 0.99999988, 0.72140878,
       0.93865955, 0.96950328, 0.91056943, 0.9497934 , 0.93196297,
       0.94721282, 0.90167129, 1.        , 0.98847109, 0.94279

In [20]:
index_df.describe()

,score
count,1000.000000
mean,0.947645
std,0.060109
min,0.554993
25%,0.924050
50%,0.963989
75%,0.993244
max,1.000000


In [23]:
index_df.sort_values('score', ascending=False)

,id_1,id_2,score
590,begin self,begin self,1.000000
588,begin self,begin self,1.000000
809,org,org,1.000000
248,begin result := integer,begin result := integer,1.000000
498,x; _y += v,x; _y += v,1.000000
...,...,...,...
975,"com> july, 2002 public domain ↑ c++ bresenham ...","c published as ""digital line drawing"" by paul ...",0.676726
802,) – 20 мл,см,0.656093
973,implementation by robert lee <rlee0001@maine,dfm} procedure tform1,0.640714
986,the original source code was in the public do...,"com> july, 2002 public domain ↑ c++ bresenham ...",0.626367


In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:

plt.figure(figsize=(10,9))#10,9
labels = flat_list[:10]
sns.heatmap(index_df[:10], xticklabels=labels, yticklabels=labels, annot=True) 


ValueError: could not convert string to float: 'в википедии имеется статья по теме «свидетельство частного пилота» гражданское пилотское свидетельство - разрешение на управление определенным видом воздушного судна'

<Figure size 720x648 with 0 Axes>